In [ ]:
#########################################################
#                      DIABETRACK                       #
#########################################################
from ClaseCita import Cita
from ClaseMedicacion import Medicacion
from ClaseSintomas import Sintomas
from Utilidades import Utilidades
from ClaseAzucar import Azucar
from ClaseComida import Comida
from ClaseInsulina import Insulina
import tkinter as tk
from datetime import datetime, date

class Diabetrack(object):
    
    def __init__(self):
        util=Utilidades()
        self.lecpacientes = util.csv_pacientes()
        self.lecmedicamentos = util.csv_medicamentos()
        self.lecazucar = util.csv_azucar()
        self.leccitas = util.csv_citas()
        self.leccomidas = util.csv_comidas()
        self.lecsintomas = util.csv_sintomas()
        self.lecrecetas = util.csv_recetas()
        self.lecinsulina = util.csv_insulina()
    
    # Esta función revisa que el usuario y la contraseña introducidas existen
    def comprueba_usuario(self,usuario):
        if usuario in self.lecpacientes:
            return self.lecpacientes[usuario] 
        else:
            return False

    def comprueba_dia(self,dia):
        dic = {}   # creamos un diccionario vacio
        for i in self.leccitas:   # recorre todoel diccionario
            if self.leccitas[i].fecha == dia:   # si la fecha de esta fila del diccionario concide con la fecha seleccionada
                dic[i] = self.leccitas[i]   # se introduce esta información al diccionario que inicialmente estaba vacio
        return dic   # devuelve el diccionario

    # Todas estas funciones de comprueba_dia funcionan igual que la comentada anteriormente

    def comprueba_dia1(self,dia):
        dic = {}
        for i in self.lecazucar:
            if self.lecazucar[i].fecha == dia:
                dic[i] = self.lecazucar[i]
        return dic

    def comprueba_dia2(self,dia):
        dic = {}
        for i in self.lecinsulina:
            if self.lecinsulina[i].fecha == dia:
                dic[i] = self.lecinsulina[i]
        return dic

    def comprueba_dia3(self,dia):
        dic = {}
        for i in self.leccomidas:
            if self.leccomidas[i].fecha == dia:
                dic[i] = self.leccomidas[i]
        return dic

    def comprueba_dia4(self,dia):
        dic = {}
        for i in self.lecsintomas:
            if self.lecsintomas[i].fecha == dia and self.lecsintomas[i].estado == 'True':
                dic[i] = self.lecsintomas[i]
        return dic

    def comprueba_dia5(self,dia):
        dic = {}
        for i in self.lecmedicamentos:
            if self.lecmedicamentos[i].fecha == dia:
                dic[i] = self.lecmedicamentos[i]
        return dic

    # Funciona como el resto de funciones que han sido creadas con el fin de meter la información en el csv
    def nueva_cita1(self,nombre, fecha, hora, doctor, ubicacion):
        cit = Cita(nombre, fecha, hora, doctor, ubicacion)
        variable = nombre + fecha + hora + doctor + ubicacion
        self.leccitas[variable] = cit
        cit1 = Cita(nombre, fecha, hora, doctor, ubicacion)
        variable1 = nombre + fecha + hora + doctor + ubicacion
        self.leccitas[variable1] = cit1
        util = Utilidades()
        lista_nombre = []
        lista_fecha = []
        lista_hora = []
        lista_doctor = []
        lista_ubi = []
        for elem in self.leccitas:
            lista_nombre.append(self.leccitas[elem].nombre)
            lista_fecha.append(self.leccitas[elem].fecha)
            lista_hora.append(self.leccitas[elem].hora)
            lista_doctor.append(self.leccitas[elem].doctor)
            lista_ubi.append(self.leccitas[elem].ubicacion)
        util.crear_citas(lista_nombre, lista_fecha, lista_hora, lista_doctor,lista_ubi)

    # Esta función está creada para volver un diccionario con los medicamentos que se toma el 
    # paciente, además, devuelve el estado de los checkbuttons que se han guardado previamente 
    # para ese día.
    def devolver_med_paciente(self):
        var = {}
        estados = []
        cont = 0
        
        for elem in self.lecmedicamentos:
            if self.lecmedicamentos[elem].hora != 'None' and self.lecmedicamentos[elem].fecha == 'None':
                var[elem] = self.lecmedicamentos[elem]
            if self.lecmedicamentos[elem].fecha == self.devolver_fecha_hoy():
                cont = 1 # Sirve para saber si ya se han creado los medicamentos para ese día o no
                estados.append(self.lecmedicamentos[elem].estado)
                
        if cont ==0: # Si no hay ninguna entrada para la fecha de hoy
            for elem in var: # Creamos un medicamento con estado 'False' para cada medicamento que se toma el paciente
                med = Medicacion(var[elem].medicamento, var[elem].dosis, var[elem].hora, 'False', self.devolver_fecha_hoy())
                self.lecmedicamentos[var[elem].medicamento + var[elem].dosis + var[elem].hora + self.devolver_fecha_hoy()] = med
                estados.append('False')
            self.actualizar_csv_med(self.lecmedicamentos) # Actualizamos el csv por si el usuario sale
        return var, estados
    
    # Esta función devuelve un diccionario con todos los nombres de medicamentos que hemos 
    # puesto en nuestra base de datos
    def devolver_med_base(self):
        var = {}
        for elem in self.lecmedicamentos:
            if self.lecmedicamentos[elem].hora == 'None':
                var[elem] = self.lecmedicamentos[elem]
        return var
    
    
    # Esta función devuelve en un diccionario todos los síntomas que tiene guardados el paciente para poder marcar.
    # Además, también devuelve una lista con los estados que se han guardado
    def devolver_sint(self):
        var = {}
        estados = []
        cont = 0
        
        for elem in self.lecsintomas:
            if self.lecsintomas[elem].fecha == 'None':
                var[elem] = self.lecsintomas[elem]
            if self.lecsintomas[elem].fecha == self.devolver_fecha_hoy():
                cont = 1 # Mira si se han guardado los síntomas hoy
                estados.append(self.lecsintomas[elem].estado)
         
        if cont ==0: #no hay sintomas guardados para el día de hoy
            for elem in var: # Se crean todos los síntomas
                sint = Sintomas(var[elem].nombre, self.devolver_fecha_hoy(), 'False')
                self.lecsintomas[var[elem].nombre + self.devolver_fecha_hoy()] = sint
                estados.append('False')
            self.actualizar_csv_sint(self.lecsintomas)
        
        return var, estados
    
    
    # Esta función nos permite seleccionar la receta que queremos que sea leída
    # para luego ser mostrada en la interfaz gráfica
    def devolver_receta(self, numreceta):
        if numreceta in self.lecrecetas:
            return self.lecrecetas[numreceta]

    # Funciona igual que la función comprueba_dia
    def mira_dia(self,dia):
        dic = {}
        for i in self.lecazucar:
            if self.lecazucar[i].fecha == dia:
                dic[i] = self.lecazucar[i]
        return dic

    # Funciona como el resto de funciones que han sido creadas con el fin de meter la información en el csv
    def nuevo_valor1(self, valor, fecha, hora, observaciones):
        val = Azucar(valor, fecha, hora, observaciones)
        variable = valor + fecha + hora + observaciones
        self.lecazucar[variable] = val
        val1 = Azucar(valor, fecha, hora, observaciones)
        variable1 = valor + fecha + hora + observaciones
        self.lecazucar[variable1] = val1
        util = Utilidades()
        lista_valor = []
        lista_fecha = []
        lista_hora = []
        lista_observaciones = []
        for elem in self.lecazucar:
            lista_valor.append(self.lecazucar[elem].valor)
            lista_fecha.append(self.lecazucar[elem].fecha)
            lista_hora.append(self.lecazucar[elem].hora)
            lista_observaciones.append(self.lecazucar[elem].observaciones)
        util.crear_valor(lista_valor, lista_fecha, lista_hora, lista_observaciones)


    # Esta función se creó para poder guardar los medicamentos nuevos que quisiera tener
    # el usuario.
    def anadirmed_dic(self, nombre, dosis, hora):
        nombre = str(nombre)
        dosis = str(dosis)
        hora = str(hora)
        fecha = str(self.devolver_fecha_hoy())
        estado = 'False'
        # Se crea una medicación para que sea devuelta en la base del paciente
        med1 = Medicacion(nombre, dosis, hora, estado = 'None', fecha = 'None')
        variable1 = nombre + dosis + hora + 'None'
        self.lecmedicamentos[variable1]=med1
        # Se inicia el medicamento para hoy, para que pueda empezar a guardarlo
        med2 = Medicacion(nombre, dosis, hora, estado, fecha)
        variable2 = nombre + dosis + hora + fecha
        self.lecmedicamentos[variable2] = med2
        # Se guarda todo en el csv por si el usuario quiere salir de la aplicación
        self.actualizar_csv_med(self.lecmedicamentos)
    
    # Funciona de manera similar a la anadirmed_dic
    # Cuando se crea un nuevo síntoma este se guarda
    def anadirsint_dic(self, nombre):
        nombre = str(nombre)
        fecha = str(self.devolver_fecha_hoy())
        # Se crea un síntoma para que se le muestre en los checkbuttons
        sint1 = Sintomas(nombre, 'None', 'None')
        variable1 = nombre + 'None'
        self.lecsintomas[variable1] = sint1
        # Se inicia el síntoma de hoy
        sint2 = Sintomas(nombre, fecha, 'False')
        variable2 = nombre+ fecha
        self.lecsintomas[variable2] = sint2
        # Se guarda todo en el csv por si el usuario quiere salir de la aplicación
        self.actualizar_csv_sint(self.lecsintomas)    
    
    # La función añadir_com nos permite crear el diccionario para luego añadir
    # las comidas agregadas al csv.
    def anadir_com(self, nombre, hora):
        nombre = str(nombre)
        fecha = str(self.devolver_fecha_hoy())
        hora = str(hora)
        com = Comida(nombre, fecha, hora)
        variable = nombre + fecha + hora
        self.leccomidas[variable]=com
        util = Utilidades()
        lista_nombres = []
        lista_fecha = []
        lista_hora = []
        for elem in self.leccomidas:
            lista_nombres.append(self.leccomidas[elem].nombre)
            lista_fecha.append(self.leccomidas[elem].fecha)
            lista_hora.append(self.leccomidas[elem].hora)
        util.crear_comidas(lista_nombres, lista_fecha, lista_hora)
    
    # Como muchas funciones tenían que actualizar el csv creé una función que pudiesen usar todas
    def actualizar_csv_med(self, lecmedicamentos):
        self.lecmedicamentos = lecmedicamentos
        util = Utilidades()
        # Creo una lista para cada elemento, ya que el csv se crea a partir de listas en Utilidades
        lista_nombres = []
        lista_dosis = []
        lista_hora = []
        lista_fecha = []
        lista_estado = []
        for elem in self.lecmedicamentos:
            # Se van añadiendo los elementos por orden
            lista_nombres.append(self.lecmedicamentos[elem].medicamento)
            lista_dosis.append(self.lecmedicamentos[elem].dosis)
            lista_hora.append(self.lecmedicamentos[elem].hora)
            lista_fecha.append(self.lecmedicamentos[elem].fecha)
            lista_estado.append(str(self.lecmedicamentos[elem].estado))
        # Se llama a la función de utilidades
        util.crear_medicamentos(lista_nombres, lista_dosis, lista_hora, lista_fecha, lista_estado)
    
    # Como muchas funciones también tenían que actualizar el csv de síntomas, creé una función conjunta
    # de manera similar a como lo hice para los medicamentos
    def actualizar_csv_sint(self, lecsintomas):
        
        self.lecsintomas = lecsintomas
        util = Utilidades()
        # Creo una lista para cada elemento porqué el csv se crea a partir de listas
        lista_nombres = []
        lista_fechas = []
        lista_estados = []
        for elem in self.lecsintomas:
            # Se van añadiendo por orden, para que el csv tenga la estructura deseada
            lista_nombres.append(self.lecsintomas[elem].nombre)
            lista_fechas.append(self.lecsintomas[elem].fecha)
            lista_estados.append(str(self.lecsintomas[elem].estado))
        # Se llama a la función de utilidades
        util.crear_sintomas(lista_nombres, lista_fechas, lista_estados)
        
     # Para la ventana de medicación, entre otras, era importante que entrara la hora ne formato
     # hora. No encontramos ningun TimeEntry, como sí existe el DateEntry así que la creamos
    def procesar_hora(self, entrada):
        valor = entrada.get()
        # Dependiendo de la longitud de la entrada, hace unas cosas u otras
        if len(valor) ==1: 
            try:
                a = int(valor)
                if a>2: # El máximo valor aquí es de 2, ya que la máxima hora es 23:59
                    entrada.delete(0,tk.END)
                    entrada.insert(0,'')
            except:
                entrada.delete(0,tk.END)
                entrada.insert(0,'')
                    
        elif len(valor) ==2:
            try:
                a = int(valor)
                if a <=23: # El máximo valor es de 23, ya que la máxima hora es 23:59
                    entrada.delete(0,tk.END)
                    entrada.insert(0,valor+':') # Añadimos los 2 puntos para que tenga formato hora
                # En el caso de que el formato no sea el correcto elimina el último valor
                else:
                    entrada.delete(0,tk.END)
                    entrada.insert(0,valor[0])
            except:
                entrada.delete(0,tk.END)
                entrada.insert(0,valor[0])
        elif len(valor)==4:
            horas, minutos = valor.split(':') # Separamos horas de minutos
            try:
                a = int(minutos)
                if a>5: # Solo miramos los minutos, estos máximo pueden valer 59
                    entrada.delete(0,tk.END)
                    entrada.insert(0,horas +':')
            except:
                entrada.delete(0,tk.END)
                entrada.insert(0,horas +':')
            
        elif len(valor)==5:
            horas, minutos = valor.split(':')
            try:
                a = int(minutos) # No hay que comprobar este número porque van de 0-9
            except:
                entrada.delete(0,tk.END)
                entrada.insert(0,horas +':'+ minutos[0])

        elif len(valor) >=6: # Si ponen caracteres de más, se eliminan automáticamente
            entrada.delete(0,tk.END)
            entrada.insert(0,valor[:(5-len(valor))])
        
    def guardar_estado_checkbutton(self, estados_checkbuttons):
        # Los checkbuttons guardan False y True en bool, nosotras, como del csv los lee como si fueran string,
        # hacemos este cambio.
        # Esta función fue creada para síntomas, pero la hicimos más general para poder utilizarla por ejemplo en 
        # Medicación
        
        estados_guardados = ['True' if estado.get() == True else 'False' for estado in estados_checkbuttons]
        return estados_guardados
        
    def devolver_fecha_hoy(self):
        
        data = date.today() # El formato que nos devuelve no es el que queremos
        return data.strftime('%d/%m/%Y')

    # Esta función actualiza el estado de la medicación para que se quede guardado para cuando se quiera
    # volver a abrir la aplicación
    def actualizar_estado_med(self, estados_guardados):
        i = 0

        for elem in self.lecmedicamentos:
            if self.lecmedicamentos[elem].fecha == self.devolver_fecha_hoy():
                med = Medicacion(self.lecmedicamentos[elem].medicamento, self.lecmedicamentos[elem].dosis, self.lecmedicamentos[elem].hora, estados_guardados[i], self.lecmedicamentos[elem].fecha)
                self.lecmedicamentos[elem]=med

                i +=1 # El contador nos permite recorrer la lista de estados
        self.actualizar_csv_med(self.lecmedicamentos)
    
    # Para poder editar los medicamentos, hacemos estos if
    def editar_med(self, nombre, dosis, hora):
        for elem in self.lecmedicamentos:
            # Lo editamos de la lista de medicamentos de base
            if self.lecmedicamentos[elem].medicamento == nombre and self.lecmedicamentos[elem].hora != 'None' and self.lecmedicamentos[elem].fecha == 'None':
                med = Medicacion(self.lecmedicamentos[elem].medicamento, dosis, hora, 'False', 'None')
                self.lecmedicamentos[elem] = med
            # Lo editamos para el registro que se crea nada más abrir la pestaña de medicamentos, para el día de hoy
            if self.lecmedicamentos[elem].medicamento == nombre and self.lecmedicamentos[elem].fecha == self.devolver_fecha_hoy():
                estado = self.lecmedicamentos[elem].estado
                med = Medicacion(self.lecmedicamentos[elem].medicamento, dosis, hora, estado, self.devolver_fecha_hoy())
                self.lecmedicamentos[elem] = med
        self.actualizar_csv_med(self.lecmedicamentos) # Llamamos la función para crear las listas para el csv
    
    # Para poder editar los síntomas hacemos esta función
    def editar_sint(self, antiguo, nombre):

        for elem in self.lecsintomas:
            # Comprobamos que el elemento del diccionario tenga el nombre antiguo y lo rescribimos
            if self.lecsintomas[elem].nombre == antiguo and self.lecsintomas[elem].fecha == 'None':
 
                sint = Sintomas(nombre, 'None', 'None')
                self.lecsintomas[elem]= sint
            # También lo hacemos para la entrada de hoy.
            # Guardando con el nombre antiguo los otros días para que no se pierda la información
            if self.lecsintomas[elem].nombre == antiguo and self.lecsintomas[elem].fecha == self.devolver_fecha_hoy():

                sint = Sintomas(nombre, self.lecsintomas[elem].fecha, self.lecsintomas[elem].estado)
                self.lecsintomas[elem] = sint
               
        self.actualizar_csv_sint(self.lecsintomas) # Llamamos la función para crear las listas para el csv
    
    # Se elimina el síntoma seleccionado
    def eliminar_sint(self, nombre):
        
        for elem in self.lecsintomas:
            
            if self.lecsintomas[elem].nombre == nombre and self.lecsintomas[elem].fecha == 'None':
                eliminar1 = elem
                
            if self.lecsintomas[elem].nombre == nombre and self.lecsintomas[elem].fecha == self.devolver_fecha_hoy():
                eliminar2 = elem
        # Como se puede cambiar el nombre del síntoma y luego darle a eliminar, lo eliminamos dentro de un try
        try:
            del self.lecsintomas[eliminar1]
            del self.lecsintomas[eliminar2]
        except:
            pass
        self.actualizar_csv_sint(self.lecsintomas) # Llamamos la función para crear las listas para el csv
    
    def eliminar_med(self, nombre):
        # Como el nombre en este caso no se puede editar, siempre se va a poder eliminar
        for elem in self.lecmedicamentos:
            # Lo eliminamos del diccionario para crear los checkbuttons
            if self.lecmedicamentos[elem].medicamento == nombre and self.lecmedicamentos[elem].hora != 'None' and self.lecmedicamentos[elem].fecha == 'None':
                eliminar1 = elem
            # Lo eliminamos de la entrada de hoy
            if self.lecmedicamentos[elem].medicamento == nombre and self.lecmedicamentos[elem].fecha == self.devolver_fecha_hoy():
                eliminar2 = elem

        del self.lecmedicamentos[eliminar1]
        del self.lecmedicamentos[eliminar2]

        self.actualizar_csv_med(self.lecmedicamentos) # Llamamos la función para crear las listas para el csv
    
    # Sirve para guardar los nuevos estados de los síntomas. Automáticamente se guardan en el csv por si el paciente quiere
    # cerrar la aplicación
    def actualizar_estado_sint(self, estados_guardados):
        i = 0

        for elem in self.lecsintomas:
            if self.lecsintomas[elem].fecha == self.devolver_fecha_hoy():
                sint = Sintomas(self.lecsintomas[elem].nombre, self.lecsintomas[elem].fecha, estados_guardados[i])
                self.lecsintomas[elem]=sint
                i +=1
        self.actualizar_csv_sint(self.lecsintomas) # Llamamos la función para crear las listas para el csv
        
        
    # Funciona como el resto de funciones que han sido creadas con el fin de meter la información en el csv
    def nueva_insulina(self,irapida, ilenta, fecha):
        val = Insulina(irapida, ilenta, fecha)
        variable = irapida + ilenta + fecha
        self.lecinsulina[variable] = val
        val1 = Insulina(irapida, ilenta, fecha)
        variable1 = irapida + ilenta + fecha
        self.lecazucar[variable1] = val1
        util = Utilidades()
        lista_irapida = []
        lista_ilenta= []
        lista_fecha = []
        for elem in self.lecinsulina:
            lista_irapida.append(self.lecinsulina[elem].irapida)
            lista_ilenta.append(self.lecinsulina[elem].ilenta)
            lista_fecha.append(self.lecinsulina[elem].fecha)
        util.crear_insulina(lista_irapida, lista_ilenta, lista_fecha)